In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import datetime
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 8)
sns.set_style("whitegrid")

In [ ]:
IMAGE_PATH = r"../data/diagrams"
SAVE_IMAGES = True

if SAVE_IMAGES:
    import os
    if not os.path.exists(IMAGE_PATH): os.makedirs(IMAGE_PATH)

## Read data

In [ ]:
rico_dateparser = lambda x: pd.to_datetime(x, format="%B %d, %Y", errors='coerce')

df_app_details = pd.read_csv('../data/app_details.csv', parse_dates=["Date Updated"], date_parser=rico_dateparser)
df_app_details.drop("Icon URL", axis=1, inplace=True)

#Fix error in data which prevents automatic float parsing
df_app_details["Number of Ratings"] = pd.to_numeric(df_app_details["Number of Ratings"], errors='coerce')

for column in ["Category","Number of Downloads","Average Rating"]:
    df_app_details[column] = df_app_details[column].astype('category')

df_app_details.rename(columns={'App Package Name': 'package_name'}, inplace=True)
df_app_details.columns = [column.replace(" ","_").lower() for column in df_app_details.columns]

#df_app_details.info(verbose=True)

In [ ]:
df_analysis = pd.read_csv('../data/mass_evaluations.csv')

categories = [column for column in df_analysis if column.endswith("_meaning") or column.endswith("_evaluation")]
categories.append("package_name")
for column in categories:
    df_analysis[column] = df_analysis[column].astype('category')
  
#df_analysis.info(verbose=True)

Dict with metric thresholds

In [ ]:
metric_data = {
    "distinct_rgb_values": [
        {"values": (None,5000), "meaning": "Less colourful", "evaluation": "normal"},
        {"values": (5000, 15000), "meaning": "Fair", "evaluation": "good"},
        {"values": (15000,), "meaning": "Colourful", "evaluation": "normal"}
    ],
    "figure_ground_contrast": [
        {"values": (None, 0.3), "meaning": "Low contrast", "evaluation": "bad"},
        {"values": (0.3, 0.7), "meaning": "Fair", "evaluation": "good"},
        {"values": (0.7,), "meaning": "High contrast", "evaluation": "bad"}
    ],
    "white_space": [
        {"values": (None, 0.3), "meaning": "Low", "evaluation": "bad"},
        {"values": (0.3, 0.8), "meaning": "Good", "evaluation": "good"},
        {"values": (0.8,), "meaning": "High", "evaluation": "bad"}
    ],
    "grid_quality": [
        {"values": (None, 100), "meaning": "Low", "evaluation": "bad"},
        {"values": (100, 220), "meaning": "Medium", "evaluation": "good"},
        {"values": (220,), "meaning": "High", "evaluation": "bad"}
    ],
    "colourfulness": [
        {"values": (None, 50.00), "meaning": "Less colourful", "evaluation": "normal"},
        {"values": (50.00, 100.00), "meaning": "Fair", "evaluation": "normal"},
        {"values": (100,), "meaning": "Colourful", "evaluation": "normal"}
    ],  
    "hsv_colours_avgSaturation": [
        {"values": (None, 0.10), "meaning": "Low", "evaluation": "bad"},
        {"values": (0.10, 0.60), "meaning": "Medium", "evaluation": "good"},
        {"values": (0.60,), "meaning": "High", "evaluation": "bad"}
    ], 
    "hsv_colours_stdSaturation": [
        {"values": (None, 0.20), "meaning": "Low", "evaluation": "bad"},
        {"values": (0.20, 0.40), "meaning": "Medium", "evaluation": "good"},
        {"values": (0.40,), "meaning": "High", "evaluation": "bad"}
    ],     
    "hsv_colours_avgValue": [
        {"values": (None, 0.40), "meaning": "Dark", "evaluation": "normal"},
        {"values": (0.40, 0.80), "meaning": "Medium", "evaluation": "normal"},
        {"values": (0.80,), "meaning": "Light", "evaluation": "bad"}
    ],  
    "hsv_colours_stdValue": [
        {"values": (None, 0.15), "meaning": "Low", "evaluation": "bad"},
        {"values": (0.15, 0.35), "meaning": "Medium", "evaluation": "good"},
        {"values": (0.35,), "meaning": "High", "evaluation": "bad"}
    ],
    "hsv_unique": [
        {"values": (None, 20000), "meaning": "Good", "evaluation": "good"},
        {"values": (20000,), "meaning": "Potential varied", "evaluation": "normal"}
    ],
    "lab_avg_meanLEvaluation": [
        {"values": (None, 40.00), "meaning": "Dark", "evaluation": "normal"},
        {"values": (40.00, 75.00), "meaning": "Medium", "evaluation": "good"},
        {"values": (75.00,), "meaning": "Light", "evaluation": "normal"}
    ], 
    "lab_avg_stdLEvaluation": [
        {"values": (None, 15.00), "meaning": "Low", "evaluation": "bad"},
        {"values": (15.00, 35.00), "meaning": "Medium", "evaluation": "good"},
        {"values": (35.00,), "meaning": "high", "evaluation": "bad"}
    ],
    "static_colour_clusters": [
        {"values": (None, 4000), "meaning": "Less colourful", "evaluation": "normal"},
        {"values": (4000, 8000), "meaning": "Fair", "evaluation": "normal"},
        {"values": (8000,), "meaning": "Colourful", "evaluation": "normal"}
    ], 
    "dynamic_colour_clusters": [
        {"values": (None, 500), "meaning": "Less colourful", "evaluation": "normal"},
        {"values": (500, 1000), "meaning": "Fair", "evaluation": "normal"},
        {"values": (1000,), "meaning": "Colourful", "evaluation": "normal"}
    ], 
    "luminance_sd": [
        {"values": (None, 60.00), "meaning": "Good", "evaluation": "good"},
        {"values": (60.00, 90.00), "meaning": "Acceptable", "evaluation": "normal"},
        {"values": (90.00,), "meaning": "Potential varied", "evaluation": "bad"}
    ], 
    "wave": [
        {"values": (None, 0.54), "meaning": "Low", "evaluation": "bad"},
        {"values": (0.54,), "meaning": "Good", "evaluation": "good"}
    ], 
    "contour_density": [
        {"values": (None, 0.12), "meaning": "Good", "evaluation": "good"},
        {"values": (0.12, 0.22), "meaning": "Fair", "evaluation": "normal"},
        {"values": (0.22,), "meaning": "Poor", "evaluation": "bad"}
    ], 
    "contour_congestion": [
        {"values": (None, 0.25), "meaning": "Good", "evaluation": "good"},
        {"values": (0.25, 0.50), "meaning": "Fair", "evaluation": "normal"},
        {"values": (0.50,), "meaning": "Poor", "evaluation": "bad"}
    ], 
    "pixel_symmetry": [
        {"values": (None, 1.00), "meaning": "Good", "evaluation": "good"},
        {"values": (1.00,), "meaning": "Poor", "evaluation": "bad"}
    ], 
    "quadtree_decomposition_balance": [
        {"values": (None, 0.65), "meaning": "Potential unbalanced", "evaluation": "bad"},
        {"values": (0.65,), "meaning": "Balanced", "evaluation": "good"}
    ], 
    "quadtree_decomposition_symmetry": [
        {"values": (None, 0.50), "meaning": "Poor", "evaluation": "bad"},
        {"values": (0.50,), "meaning": "Acceptable", "evaluation": "good"}
    ], 
    "quadtree_decomposition_equilibrium": [
        {"values": (None, 0.65), "meaning": "Not centralized", "evaluation": "bad"},
        {"values": (0.65,), "meaning": "Centralized", "evaluation": "good"}
    ],
    "quadtree_decomposition_numberOfLeaves": [
        {"values": (None, 1500), "meaning": "Good", "evaluation": "good"},
        {"values": (1500, 3200), "meaning": "Fair", "evaluation": "normal"},
        {"values": (3200,), "meaning": "Poor", "evaluation": "bad"}
    ] 
}

### Merge dataframes

In [ ]:
df = df_analysis.merge(df_app_details, on="package_name", how='left')
for column in ["package_name","play_store_name"]:
    df[column] = df[column].astype('category')

    
eval_columns = [column for column in df.columns if column.endswith("_evaluation")]
value_columns = [column for column in df.columns if column.endswith("_value")]
time_columns = [column for column in df.columns if column.endswith("_time")]

df["count_bad"] = df[eval_columns].eq("bad").sum(axis=1)
df["count_normal"] = df[eval_columns].eq("normal").sum(axis=1)
df["count_good"] = df[eval_columns].eq("good").sum(axis=1)

df.info(verbose=True)

In [ ]:
df.head()

## Analyzation

### Data Overview

In [ ]:
print("Number of data entries: %d" % len(df))
print("Number of analysed metrics per image: %d" % len(eval_columns))
print("Number of unique apps: %d" % len(df["package_name"].cat.categories))

#### Number of Screenshots per App Category

In [ ]:
sns.countplot(data=df, y="category")

### Metric analyzation

#### Number of "good", "normal" and "bad" labels for each metric evaluation

In [ ]:
rating_order = ["bad","normal","good"]

df_melt = pd.melt(df)
plt.figure(figsize=(20, 14))
sns.countplot(data=df_melt.loc[df_melt["variable"].str.contains("_evaluation")], y='variable', hue='value', hue_order=rating_order)

#### Boxplot for each metric

In [ ]:
numeric_colums = df_analysis.select_dtypes(include="number").columns
numeric_colums = [column for column in numeric_colums if column.endswith("_value")]
list_0_1 = [column for column in numeric_colums if df_analysis[column].max() <= 1]
if list_0_1: sns.boxplot(data=df_analysis[list_0_1], orient='h')

In [ ]:
list_1_500 = [column for column in numeric_colums if 1 < df_analysis[column].max() <= 500]
sns.boxplot(data=df_analysis[list_1_500], orient='h')

In [ ]:
remaining_colums = [x for x in numeric_colums if x not in list_0_1 and x not in list_1_500]
sns.boxplot(data=df_analysis[remaining_colums], orient='h')

#### Mean values of each metric

In [ ]:
df[value_columns].mean()

#### Value distribution of each metric

In [ ]:
axes_colums = 3
axes_rows = math.ceil(len(value_columns) / axes_colums)
metric_colors = {"good": "g", "normal": "y", "bad": "r"}

if SAVE_IMAGES:
    os.makedirs(IMAGE_PATH + "/metricDistribution")

fig, axes = plt.subplots(axes_rows, axes_colums, figsize=(24, 5 * axes_rows))
plt.subplots_adjust(hspace=0.25)
for idx, column in enumerate(value_columns):
    #print("%s,%s" % ( math.floor(idx / axes_colums), idx % axes_colums )  )
    ax = axes[math.floor(idx / axes_colums), idx % axes_colums]
    g = sns.histplot(ax=ax, data=df, x=column, bins=30)

    data_key = column[:-len("_value")]
    if data_key in metric_data:
        data = metric_data[column[:-len("_value")]]
        for area in data:
            area_min = area["values"][0]
            if not area_min:
                area_min = ax.get_xlim()[0]
            area_max = None
            if(len(area["values"]) > 1): 
                area_max = area["values"][1]
            else: 
                area_max = ax.get_xlim()[1]
            if area_min > area_max: area_min = area_max

            g.axvspan(area_min, area_max, color=metric_colors[area["evaluation"]], alpha=0.3, zorder=-100)
            g.text(area_max, ax.get_ylim()[1], area["meaning"], horizontalalignment='right', verticalalignment='center')

    if SAVE_IMAGES:
        extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())  
        fig.savefig(IMAGE_PATH + "/metricDistribution/%s.png" % column, bbox_inches=extent.expanded(1.25, 1.25).translated(-0.12,-0.18))

g

## Per Screenshot and App analysis

#### Number of metric evaluation labels grouped by app

In [ ]:
df_counts = df.groupby('package_name')[['count_bad','count_normal','count_good']].sum()
df_counts['total'] = df_counts.sum(axis=1)
df_counts['package_percentage_bad'] = df_counts['count_bad'] / df_counts['total'] * 100
df_counts['package_percentage_normal'] = df_counts['count_normal'] / df_counts['total'] * 100
df_counts['package_percentage_good'] = df_counts['count_good'] / df_counts['total'] * 100
df_counts.reset_index(inplace=True)

df_counts = df_counts.loc[:,['package_name','package_percentage_bad','package_percentage_normal','package_percentage_good']]
df = df.merge(df_counts, on="package_name", how='left')

df_counts.head(10)

#### Percentage good and bad counts in relation to app rating

In [ ]:
df_packages = df[['package_name','package_percentage_bad','package_percentage_normal','package_percentage_good','average_rating','number_of_ratings']].drop_duplicates(subset=['package_name'])

df_packages = df_packages[df_packages['number_of_ratings'] >= 1000]
df_packages['package_percentage_bad'] = pd.cut(df_packages['package_percentage_bad'], 10)
df_packages['package_percentage_normal'] = pd.cut(df_packages['package_percentage_normal'], 10)
df_packages['package_percentage_good'] = pd.cut(df_packages['package_percentage_good'], 10)
df_packages.reset_index(inplace=True)


df_packages_good = df_packages[['average_rating','package_percentage_good']].value_counts().reset_index(name='count')
df_packages_good = pd.pivot_table(df_packages_good, index='package_percentage_good', columns='average_rating', values='count')
df_packages_good.sort_index(ascending=False, inplace=True)

sns.heatmap(data=df_packages_good, annot=True)

In [ ]:
df_packages_bad = df_packages[['average_rating','package_percentage_bad']].value_counts().reset_index(name='count')
df_packages_bad = pd.pivot_table(df_packages_bad, index='package_percentage_bad', columns='average_rating', values='count')
df_packages_bad.sort_index(ascending=True, inplace=True)

sns.heatmap(data=df_packages_bad, annot=True)

#### Best and worst of all images

In [ ]:
top_20_best = df[['image', 'count_good']].nlargest(20, 'count_good')
sns.barplot(data=top_20_best[:10], x='image', y='count_good')

In [ ]:
top_20_worst = df[['image', 'count_bad']].nlargest(20, 'count_bad')
sns.barplot(data=top_20_worst[:10], x='image', y='count_bad')

#### Best and worst images rendered

In [ ]:
data = np.array([top_20_best, top_20_worst])

f, axarr = plt.subplots(2,5,figsize=(24,16))
axarr[0][2].set_title("Top 5 Best")
axarr[1][2].set_title("Top 5 Worst")
for i in range(0,len(axarr)):
    for j in range(0,len(axarr[0])):
        axarr[i][j].grid(False)
        axarr[i][j].set_xticks([])
        axarr[i][j].set_yticks([])

        img_name = data[i][j][0]
        axarr[i][j].imshow(mpimg.imread('../data/combined/%s' % img_name))

        if i == 0:
            axarr[i][j].set_xlabel("#%s: %s" % (j + 1, img_name))
        else:
            axarr[i][j].set_xlabel("#%s: %s" % (j + 1, img_name))


#### Metrics for best image

In [ ]:
def create_df_one_pic(image_name):
    df_metric_per_pic = {'metric': [], 'value': [], 'evaluation': [], 'meaning': []}

    for key, value in df[df["image"] == image_name].to_dict('records')[0].items():
        if key.endswith("_value"): 
            plain_name = key[:-len("_value")]
            df_metric_per_pic['metric'].append(plain_name)
            df_metric_per_pic['value'].append(value)

        if key.endswith("_evaluation"): df_metric_per_pic['evaluation'].append(value)
        if key.endswith("_meaning"): df_metric_per_pic['meaning'].append(value)

    return pd.DataFrame(df_metric_per_pic)

create_df_one_pic(top_20_best.iloc[0][0])

#### Metrics for worst image

In [ ]:
create_df_one_pic(top_20_worst.iloc[0][0])

#### Top 5 to 15

In [ ]:
data = np.array([top_20_best[5:], top_20_worst[5:]])

f, axarr = plt.subplots(2,15,figsize=(24,6))
axarr[0][7].set_title("Top 5-15 Best")
axarr[1][7].set_title("Top 5-15 Worst")
for i in range(0,len(axarr)):
    for j in range(0,len(axarr[0])):
        axarr[i][j].grid(False)
        axarr[i][j].set_xticks([])
        axarr[i][j].set_yticks([])

        img_name = data[i][j][0]
        axarr[i][j].imshow(mpimg.imread('../data/combined/%s' % img_name))

        if i == 0:
            axarr[i][j].set_xlabel("#%s: %s" % (j + 6, img_name))
        else:
            axarr[i][j].set_xlabel("#%s: %s" % (j + 6, img_name))

### Execution times on the server machine
On a linux VM with two worker threads (100% CPU load on both physical CPUs)

In [ ]:
print("Total execution time for one image: %s" % datetime.timedelta(seconds=df[time_columns].mean().sum()))

In [ ]:
df[time_columns].mean().sort_values(ascending=False).map(lambda x: str(datetime.timedelta(seconds=x)))

#### Highest execution times for metric "dynamic_colour_clusters_time"

In [ ]:
m = "dynamic_colour_clusters_time"
largest = df[m].nlargest(7)
data = df[df[m] >= largest.values[-1]][["image",m]].sort_values(by=m, ascending=False).values

f, axarr = plt.subplots(1,7,figsize=(24,10))
axarr[3].set_title("Longest execution times for %s" % m)
for i in range(0,len(axarr)):
        axarr[i].grid(False)
        axarr[i].set_xticks([])
        axarr[i].set_yticks([])

        img_name = data[i][0]
        axarr[i].imshow(mpimg.imread('../data/combined/%s' % img_name))
        axarr[i].set_xlabel("#%s: %s\nTime: %s" % (i + 1, img_name, datetime.timedelta(seconds=data[i][1])))